# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The Apache Cassandra coding portion. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try: 
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS music_db 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try: 
    session.set_keyspace('music_db')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## <font color=red> Query 1</font>  
**Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**

### How should we model this data? What should be our Primary Key and Partition Key? Since our data is looking for the *`SessionId`* and *`itemInSession`* let's start with that. That should be enough to make the row key unique.

`Table Name: music_history
column 1: session_id
column 2: item_in_session
column 3: artist_name
Column 4: song_title
Column 5: song_length
PRIMARY KEY(session_id, item_in_session)`

In [10]:
# 1st: create the table
query = """CREATE TABLE IF NOT EXISTS music_history( 
            session_id INT, item_in_session INT, artist_name TEXT, song_title TEXT, song_length FLOAT,  
            PRIMARY KEY (session_id, item_in_session));"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [11]:
# 2nd: insert data from the csv file to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [12]:
# 3rd: Add in the SELECT statement to verify the data was entered into the table
query = """SELECT session_id, item_in_session, artist_name, song_title, song_length 
           FROM music_history 
           WHERE session_id = 338 
           AND item_in_session = 4;"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist_name, row.song_title, row.song_length)

338 4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## <font color=red> Query 2</font>  
**Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

### How should we model this data? What should be our Primary Key and Partition Key? Since our data is looking for the *`userid`* and *`sessionid`* let's start with that.  We also need to add the `itemInSession` as a Clustering Column to sort the data.

`Table Name: song_playlist_session
column 1: user_id
column 2: session_id
column 3: item_in_session
Column 4: artist_name
Column 5: song_title
Column 6: user_firstname
Column 7: user_lastname
PRIMARY KEY((user_id, session_id), item_in_session)`

In [13]:
# 1st: create the table
query = """CREATE TABLE IF NOT EXISTS song_playlist_session( 
            user_id INT, session_id INT, item_in_session INT, artist_name TEXT, song_title TEXT, 
            user_firstname TEXT, user_lastname TEXT,
            PRIMARY KEY ((user_id, session_id), item_in_session)
            ) WITH CLUSTERING ORDER BY (item_in_session DESC);"""
try:
    session.execute(query)
except Exception as e:
    print(e)  
                    

In [14]:
# 2nd: insert data from the csv file to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_playlist_session 
                   (user_id, session_id, item_in_session, artist_name, song_title, user_firstname, user_lastname)
                   VALUES (%s, %s, %s, %s, %s, %s, %s);"""
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1], line[4]))

In [15]:
# 3rd: Add the SELECT statement to verify the data was entered into the table
query = """SELECT user_id, session_id, item_in_session, artist_name, song_title, user_firstname, user_lastname 
           FROM song_playlist_session 
           WHERE user_id = 10 
           AND session_id = 182;"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_id, row.session_id, row.item_in_session, row.artist_name, row.song_title, row.user_firstname, \
          row.user_lastname)

10 182 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
10 182 2 Sebastien Tellier Kilometer Sylvie Cruz
10 182 1 Three Drives Greece 2000 Sylvie Cruz
10 182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz


## <font color=red> Query 3</font>  
**Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**


### How should we model this data? What should be our Primary Key and Partition Key? Since our data is looking for the *`Song`* let's start with that. We also need to add the `userID` as a Clustering Column, as multiple users may listen to the same song.That should be enough to make the row key unique.

`Table Name: song_listeners
column 1: song_title
column 2: user_id
column 3: user_firstname
column 4: user_lastname
PRIMARY KEY((song_title),user_id)`

In [16]:
# 1st: create the table
query = """CREATE TABLE IF NOT EXISTS song_listeners( 
            song_title TEXT, user_id INT, user_firstname TEXT, user_lastname TEXT, 
            PRIMARY KEY (song_title));"""
try:
    session.execute(query)
except Exception as e:
    print(e)  

                    

In [19]:
# 2nd: insert data from the csv file to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listeners (song_title, user_id, user_firstname, user_lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [20]:
# 3rd: Add in the SELECT statement to verify the data was entered into the table
query = """SELECT song_title, user_id,user_firstname, user_lastname 
           FROM song_listeners 
           WHERE song_title = 'All Hands Against His Own';"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title, row.user_firstname, row.user_lastname)

All Hands Against His Own Jacqueline Lynch


### Drop the tables before closing out the sessions

In [21]:
# Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE IF EXISTS song_playlist_session"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_listeners"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [22]:
session.shutdown()
cluster.shutdown()